In [36]:
import pandas as pd
import sqlalchemy as sqla

In [2]:
reviews = pd.read_csv('src/reviews.csv')

# checking if reviews exist
reviews.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [21]:
# seperate the positive and negative reviews with each other. also rename all the columns
negative_df = reviews.loc[:,["Negative_Review", "Review_Total_Negative_Word_Counts"]]
negative_df["positive"] = 0
negative_df = negative_df.rename(columns={
    "Negative_Review": "review",
    "Review_Total_Negative_Word_Counts": "word_count"
})

positive_df = reviews.loc[:,["Positive_Review", "Review_Total_Positive_Word_Counts"]]
positive_df["positive"] = 1
positive_df = positive_df.rename(columns={
    "Positive_Review": "review",
    "Review_Total_Positive_Word_Counts": "word_count"
})

reviews_df = pd.concat([negative_df, positive_df]).reset_index(drop=True)
reviews_df.tail(10)

,review,word_count,positive
1031466,Nothing,2,1
1031467,The breakfast was excelent lots of fress frui...,27,1
1031468,large room,3,1
1031469,Good breakfast,3,1
1031470,helpful staff allowed me to check in early as...,18,1
1031471,location,2,1
1031472,Breakfast was ok and we got earlier check in,11,1
1031473,No Positive,0,1
1031474,The rooms are enormous and really comfortable...,25,1
1031475,staff was very kind,6,1


In [31]:
no_empty = reviews_df
# see if works
no_empty.head(3)

,review,word_count,positive
0,I am so angry that i made this post available...,397,0
1,No Negative,0,0
2,Rooms are nice but for elderly a bit difficul...,42,0


In [34]:
# Remove the empty reviews
no_empty = no_empty[no_empty.word_count != 0]
# reset the index
no_empty = no_empty.reset_index(drop=True)
no_empty

,review,word_count,positive
0,I am so angry that i made this post available...,397,0
1,Rooms are nice but for elderly a bit difficul...,42,0
2,My room was dirty and I was afraid to walk ba...,210,0
3,You When I booked with your company on line y...,140,0
4,Backyard of the hotel is total mess shouldn t...,17,0
...,...,...,...
867635,helpful staff allowed me to check in early as...,18,1
867636,location,2,1
867637,Breakfast was ok and we got earlier check in,11,1
867638,The rooms are enormous and really comfortable...,25,1


In [ ]:
# The procedure that's being used to insert data into the table

# DELIMITER //

# CREATE PROCEDURE addReview 
# (
#    	param_content TEXT,
#    	param_count INT,
# 	param_positive BOOLEAN
# ) 
# BEGIN 
# 	INSERT INTO reviews(content, count, positive)
#     VALUES(param_content, param_count, param_positive);
# END//

# DELIMITER ;

In [37]:
# engine = mysql
# user = root
# password = ''
# url = 127.0.0.1
# database = hotel-reviews
# engine = sqla.create_engine('mysql://root:@127.0.0.1/hotel-reviews?charset=utf8')

In [76]:
def call_procedure(function_name, params):
    
    connection = sqla.create_engine('mysql://root:@localhost/hotel-reviews?charset=utf8').raw_connection()
    try:
        cursor = connection.cursor()
        cursor.callproc(function_name, params)
        results = list(cursor.fetchall())
        cursor.close()
        connection.commit()
        return results
    finally:
        connection.close()
        
# call_procedure('addReview', ['Simon' , 1 ,1])

# Testing if the stored procedure works
# with engine.connect() as session:
#     session.execute('addReview :c, :w, :p', {'c': 'Simon', 'w': 1, 'p': True})

In [58]:
for _i, _v in no_empty.iterrows():
    call_procedure('addReview', [_v['review'], _v['word_count'], _v['positive']])


OperationalError: (MySQLdb._exceptions.OperationalError) (2003, "Can't connect to MySQL server on '127.0.0.1' (10048)")
(Background on this error at: http://sqlalche.me/e/e3q8)

In [65]:
no_empty.iloc[308903]

len(no_empty)

867640

In [77]:
# len(no_empty[308903:])
continue_no_empty = no_empty[341298:]
# continue_no_empty.iloc[0]
for _i, _v in continue_no_empty.iterrows():
    call_procedure('addReview', [_v['review'], _v['word_count'], _v['positive']])

c:\users\simon\appdata\local\programs\python\python37\lib\site-packages\sqlalchemy\engine\default.py:356: SAWarning: Exception attempting to detect unicode returns: OperationalError("(MySQLdb._exceptions.OperationalError) (2013, 'Lost connection to MySQL server during query')")
  "detect unicode returns: %r" % de
c:\users\simon\appdata\local\programs\python\python37\lib\site-packages\sqlalchemy\engine\default.py:356: SAWarning: Exception attempting to detect unicode returns: OperationalError('(MySQLdb._exceptions.OperationalError) (2003, "Can\'t connect to MySQL server on \'localhost\' (10061)")')
  "detect unicode returns: %r" % de


OperationalError: (MySQLdb._exceptions.OperationalError) (2003, "Can't connect to MySQL server on 'localhost' (10061)")
(Background on this error at: http://sqlalche.me/e/e3q8)